<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#导包" data-toc-modified-id="导包-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>导包</a></span></li><li><span><a href="#从腾讯疫情界面调取数据接口，得到疫情历史数据" data-toc-modified-id="从腾讯疫情界面调取数据接口，得到疫情历史数据-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>从腾讯疫情界面调取数据接口，得到疫情历史数据</a></span></li><li><span><a href="#数据清洗" data-toc-modified-id="数据清洗-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>数据清洗</a></span></li></ul></div>

### 导包

In [13]:
import json,time,requests

import numpy as np
import pandas as pd

### 从腾讯疫情界面调取数据接口，得到疫情历史数据

In [14]:
url = 'https://view.inews.qq.com/g2/getOnsInfo?name=disease_other&callback=jQuery341026745307075030955_1584946267054&_=1584946267055'
headers ={
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.25 Safari/537.36 Core/1.70.3741.400 QQBrowser/10.5.3863.400'
}
res = requests.get(url,headers=headers)
response_data = json.loads(res.text.replace('jQuery341026745307075030955_1584946267054(','')[:-1])


### 数据清洗

In [15]:
data = json.loads(response_data['data'])
a = []
b = []
c = []
d = []
e = []

# 遍历数据，取出目标数据（时间，治愈人数，死亡人数）
for x in data['dailyHistory']:
    a.append(x['date'])
    c.append(x['country']['heal'])
    d.append(x['country']['dead'])

# 遍历取出累计确诊认识
for x in data['chinaDayList']:
    b.append(x['date'])
    e.append(x['confirm'])

# 结果保存成Dataframe    
result = pd.DataFrame({'date':a,'heal':c, 'dead':d})

result1 = pd.DataFrame({'date':b,'confirm':e})
result1 = result1.iloc[:-1]

# 把confirm按照时间匹配
result2 = result.merge(result1,left_on='date',right_on='date')

In [16]:
# 获取累计确诊人数
new_confirm = result2['confirm'].diff().fillna(0)

# 获取累计治愈人数
new_heal = result2['heal'].diff().fillna(0)

In [17]:
# 添加到总数据Dataframe并重命名为finall_data
result2 ['new_confirm'] = new_confirm
result2 ['new_heal'] = new_heal
finall_data = result2

In [18]:
finall_data

,date,heal,dead,confirm,new_confirm,new_heal
0,01.20,25,6,291,0.0,0.0
1,01.23,34,25,835,544.0,9.0
2,01.24,38,41,1297,462.0,4.0
3,01.25,49,56,1985,688.0,11.0
4,01.26,51,80,2761,776.0,2.0
...,...,...,...,...,...,...
112,05.14,79644,4644,84469,5.0,23.0
113,05.15,79668,4644,84478,9.0,24.0
114,05.16,79682,4645,84484,6.0,14.0
115,05.17,79701,4645,84494,10.0,19.0


In [19]:
# 把数据保存成csv文件
# finall_data.to_csv('./virus.csv')